<h2 style = "text-align: center; font-weight: 700">Variable Neighbourhood Descent for SAT Problem</h2>

In [1]:
from copy import deepcopy

#### Heuristic Function
###### Operates on global variables declared above for effective data sharing for the exec function

In [2]:
def heuristic(curr_state, clause_list):
  hn = 0
  ans = None
  loc_dict = {"ans": ans, "curr_state": curr_state}           
  #! ↑ The exec function will only be able to modify these two variables and nothing else as it should for max. security

  for i in range(len(clause_list)):
    exec(clause_list[i], {}, loc_dict)                  #* Working! # no globals and local_dict with 2 required variables
    ans = loc_dict["ans"]                               # changing local ans to refer to what exec's ans was
    if ans == True:          
      hn += 1
      
  return hn

### Movegen Functions

#### 1-bit Flip

In [3]:
def one_bit_flip_perms(curr_state):
  new_states = []
  for i in range(len(curr_state)):
    new_states.append(deepcopy(curr_state))
    new_states[-1][i] = not new_states[-1][i]
    
  return new_states

#### 2-bit Flip

In [4]:
def two_bit_flip_perms(curr_state):
  gap = 1
  new_states = []
  while True:
    i = 0
    j = i + gap
    while i < len(curr_state) - gap:
      new_states.append(deepcopy(curr_state))
      new_states[-1][i] = not(new_states[-1][i])
      new_states[-1][j] = not(new_states[-1][j])
      i += 1
      j = i + gap
    gap += 1
    if gap > len(curr_state) - 1:
      break
  
  return new_states

#### not 2-bit Flip (3-bits flip in case of 5 literals)

In [5]:
def not_2_bit_flip_perms(curr_state):
  gap = 1
  new_states = []
  while True:
    i = 0
    j = i + gap
    while i < len(curr_state) - gap:
      new_states.append(deepcopy(curr_state))
      for t in range (len(new_states[-1])):
        new_states[-1][t] = not(new_states[-1][t])
      new_states[-1][i] = not(new_states[-1][i])
      new_states[-1][j] = not(new_states[-1][j])
      i += 1
      j = i + gap
    gap += 1
    if gap > len(curr_state) - 1:
      break
  
  return new_states

#### not 1-bit Flip (4-bits flip in case of 5 literals)

In [6]:
def not_1_bit_flips(curr_state):
  new_states = []
  for i in range(len(curr_state)):
    new_states.append(deepcopy(curr_state))
    for t in range (len(new_states[-1])):
        new_states[-1][t] = not(new_states[-1][t])
    new_states[-1][i] = not new_states[-1][i]
    
  return new_states

#### All-bits flip (5-bits flip in case of 5 literals)

In [7]:
def all_flip(curr_state):
  return [[not(i) for i in curr_state]]

#### Utility FinalAns print function

In [8]:
def summary_print(all_paths):
  for i in all_paths:
    print(f"\033[38;5;44mFor {i[-1]},\033[0m")
    for j in i[0]:
      if i[0].index(j) != len(i[0]) - 1:            # to avoid the last bottom arrow
        print("|", *j[0], "| h(n) = ", j[1], end = "\n\t\t↓\n")
      else:
        print("|", *j[0], "| h(n) = ", j[1], end = "\n\n")

#### <-- Main --> Variable Neighbourhood Descent for SAT (satisfiabillity) problem

In [9]:
def SAT_VND(start_state, clause_list):
  all_paths = []
  movegen_funlist = [one_bit_flip_perms, two_bit_flip_perms, not_2_bit_flip_perms, not_1_bit_flips, all_flip]

  # 5 movegens
  for i in range(len(movegen_funlist)):
    node = [start_state, heuristic(start_state, clause_list)]
    path = []

    while True:
      if node[1] != len(clause_list):
        new_states = movegen_funlist[i](node[0])
        for j in range(len(new_states)):
          new_states[j] = [new_states[j], heuristic(new_states[j], clause_list)]
        new_states.sort(key = lambda x: x[1], reverse = True)
        new_node = new_states[0]
        if new_node[1] > node[1]:
          path.append(node)           
          node = new_node
        else:
          break
      
      else:
        break
    if(node[1] == len(clause_list)):
      path.append(node)
      all_paths.append([path, f"{i + 1}-bit flip Movegen"])     
    else:
      print(f"\033[38;5;197mFailed to find a path Current state to the Satisfied state using {i + 1}-bit flips\033[0m")
  
  summary_print(all_paths)


#### Callers

<p style="font-size: 16px;"> Logical sentence is stored in a form of list of strings with each string being a seperate clause </p>

##### F = (A V ~B) ^ (B V ~C) ^ (~B) ^ (~C V E) ^ (A V C) ^ (~C V ~D)

In [10]:
clause_list = ["ans = (curr_state[0] or not curr_state[1])", "ans = (curr_state[1] or not curr_state[2])", "ans = (not curr_state[1])", "ans = (not curr_state[2] or curr_state[4])", "ans = (curr_state[0] or curr_state[2])", "ans = (not curr_state[2] or not curr_state[2])"]
SAT_VND([True for i in range(5)], clause_list)

Failed to find a path Current state to the Satisfied state using 5-bit flips
For 1-bit flip Movegen,
| True True True True True | h(n) =  4
		↓
| True True False True True | h(n) =  5
		↓
| True False False True True | h(n) =  6

For 2-bit flip Movegen,
| True True True True True | h(n) =  4
		↓
| True False False True True | h(n) =  6

For 3-bit flip Movegen,
| True True True True True | h(n) =  4
		↓
| True False False True False | h(n) =  6

For 4-bit flip Movegen,
| True True True True True | h(n) =  4
		↓
| True False False False False | h(n) =  6



##### F = ( A V B) ^ ( A ^ ~C) ^ ( B ^ D ) ^ ( A V ~E)

In [11]:
clause_list = ["ans = (curr_state[0] or curr_state[1])", "ans = (curr_state[0] and not curr_state[2])", "ans = (not curr_state[1] and curr_state[3])", "ans = (curr_state[0] or not curr_state[4])"]
SAT_VND([True for i in range(5)], clause_list)

Failed to find a path Current state to the Satisfied state using 4-bit flips
Failed to find a path Current state to the Satisfied state using 5-bit flips
For 1-bit flip Movegen,
| True True True True True | h(n) =  2
		↓
| True False True True True | h(n) =  3
		↓
| True False False True True | h(n) =  4

For 2-bit flip Movegen,
| True True True True True | h(n) =  2
		↓
| True False False True True | h(n) =  4

For 3-bit flip Movegen,
| True True True True True | h(n) =  2
		↓
| True False False True False | h(n) =  4

